<a href="https://colab.research.google.com/github/Regina-Arthur/Coding-Practice-Projects/blob/main/Pytorch_tutorial/RNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load CSV
df = pd.read_csv("/content/drive/MyDrive/traffic_prediction_dataset/traffic.csv")

print(df.head())

              DateTime  Junction  Vehicles           ID
0  2015-11-01 00:00:00         1        15  20151101001
1  2015-11-01 01:00:00         1        13  20151101011
2  2015-11-01 02:00:00         1        10  20151101021
3  2015-11-01 03:00:00         1         7  20151101031
4  2015-11-01 04:00:00         1         9  20151101041


In [ ]:
# Convert to datetime
df['datetime'] = pd.to_datetime(df['DateTime'])

# Extract day of week
df['day_of_week'] = df['datetime'].dt.day_name()
df['day_of_week_num'] = df['datetime'].dt.day_of_week

# Extract time (hour and minute)
df['hour'] = df['datetime'].dt.hour

df['minute'] = df['datetime'].dt.minute


# Optional: keep only those columns
df = df[['day_of_week', 'day_of_week_num', 'hour', 'minute', 'Junction', 'Vehicles']]

print(df.head())

  day_of_week  day_of_week_num  hour  minute  Junction  Vehicles
0      Sunday                6     0       0         1        15
1      Sunday                6     1       0         1        13
2      Sunday                6     2       0         1        10
3      Sunday                6     3       0         1         7
4      Sunday                6     4       0         1         9


In [ ]:
# Normalize hour
mean =df['hour'].mean()
std =df['hour'].std()
df['hour_norm'] = (df['hour']- mean)/std

mean =df['day_of_week_num'].mean()
std =df['day_of_week_num'].std()
df['day_of_week_num_norm'] = (df['day_of_week_num']- mean)/std

mean =df['Vehicles'].mean()
std =df['Vehicles'].std()
df['vehicles_norm'] = (df['Vehicles']- mean)/std

mean =df['Junction'].mean()
std =df['Junction'].std()
df['junction_norm'] = (df['Junction']- mean)/std

# Optional: keep only those columns
df = df[['day_of_week_num_norm', 'hour_norm', 'junction_norm', 'vehicles_norm']]

print(df.head())

   day_of_week_num_norm  hour_norm  junction_norm  vehicles_norm
0              1.501982  -1.661308      -1.220893      -0.375485
1              1.501982  -1.516846      -1.220893      -0.471870
2              1.501982  -1.372384      -1.220893      -0.616448
3              1.501982  -1.227923      -1.220893      -0.761026
4              1.501982  -1.083461      -1.220893      -0.664641


In [ ]:
import numpy as np

# Select features (inputs)
features = ['day_of_week_num_norm', 'hour_norm', 'junction_norm', 'vehicles_norm']
data = df[features].values.astype(np.float32)

# Sequence length
seq_len = 1
X, y = [], []

for i in range(len(data) - seq_len):
    X.append(data[i:i+seq_len])      # 24 hours of data
    y.append(data[i+seq_len, -1])    # next hour's value

X = np.array(X)
y = np.array(y).reshape(-1, 1)

print(X.shape, y.shape)  # e.g. (N, 24, 4), (N, 1)

(48119, 1, 4) (48119, 1)


In [ ]:
import torch

X = torch.tensor(X)
y = torch.tensor(y)

print(X.shape, y.shape)

X_train = X[:40000]
X_test = X[40001:]
print(X_train.shape, X_test.shape)

y_train = y[:40000]
y_test = y[40001:]
print(y_train.shape, y_test.shape)

torch.Size([48119, 1, 4]) torch.Size([48119, 1])
torch.Size([40000, 1, 4]) torch.Size([8118, 1, 4])
torch.Size([40000, 1]) torch.Size([8118, 1])


/tmp/ipython-input-2621937081.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)
/tmp/ipython-input-2621937081.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y)
